# scikit-learn for NLP -- Part 2 Feature Engineering


If you are reading this, that means you are still alive. Welcome back to the reality of learning scikit-learn. 

This tutorial focuses on feature engineering and covers more advanced topics in scikit-learn like feature extraction, building a pipeline, creating custom transformers, feature union, dimensionality reduction, etc. Feature engineering is a very important step in NLP and ML; it is not a trival task to select good features. Therefore, we are spending a lot of time on it here. 

Without further ado, let's start with loading a dataset again. This time, we will use a CSV file that has more than two columns, i.e. one column for labels and mutiple columns for raw data/features. This time, we are using a subset of the Yelp Review Data Challenge dataset. Just like the 20 News Group dataset, I converted this dataset to CSV. 

There are 5 star ratings (shocking), and I extracted 500 reviews for each rating. This dataset is small, because it's only intended for some quick demo. Other than extracing the text of each review, I also included other users' votes for each review, i.e funny, useful and cool.

In [16]:
import pandas as pd

dataset = pd.read_csv('yelp-review-subset.csv', header=0, delimiter=',', names=['stars', 'text', 'funny', 'useful', 'cool'])

# just checking the dataset
print('There are {0} star ratings, and {1} reviews'.format(len(dataset.stars.unique()), len(dataset)))
print(dataset.stars.value_counts())


There are 5 star ratings, and 2500 reviews
5    500
3    500
1    500
4    500
2    500
Name: stars, dtype: int64


With Pandas dataframe, it is very easy to select a subset of a dataframe by column names, simply pass in a list of column names. So we are going to split our data just like the previous tutorial. 

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset[['text', 'funny', 'useful', 'cool']], dataset['stars'], train_size=0.8)

The difference now is that we have four columns in our raw data. Three of them are ``funny``, ``useful`` and ``cool`` which contain numeric values, which is perfect for scikit-learn as it expects values of features to be numeric or indexes. What we need to do is to extract features from the ``text`` column. 

In [13]:
print(X_train.columns)

Index(['text', 'funny', 'useful', 'cool'], dtype='object')


We can first do something very similar to the previous tutorial, we initialize a ``CountVectorizer`` object and then pass raw text dat into its ``fit_transform()`` function to index the count of each word. Please note that you should not pass in the whole ``X_train`` dataframe into the function, but only the ``text`` column, i.e. the ``X_train.text`` dataframe (more or less like an array). Otherwise, it does not extract features from your ``text`` column, but simly indexes all the values in each column. Please see the shape of the output of two different dataframes. 

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
# initialize a CountVectorizer
cv = CountVectorizer()
# fit the raw data into the vectorizer and tranform it into a series of arrays
X_train_counts = cv.fit_transform(X_train.text)
print(X_train_counts.shape)

# this is not what you want.
cv_test = CountVectorizer()
X_train_counts_test = cv.fit_transform(X_train)
print(X_train_counts_test.shape)

(2000, 11945)
(4, 4)


Now we have a problem, the ``X_train_counts`` vector only contains features from ``text``, but now what should we do if we want to include ``funny``, ``useful`` and ``cool`` vote counts as feature as well? 

In [ ]:
from sklearn.base import TransformerMixin

class ItemSelector(TransformerMixin):
    def __init__(self, keys):
        self.keys = keys

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.keys]
    
